In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import itertools
from keras.callbacks import EarlyStopping
from keras import backend as K
from sklearn.metrics import f1_score
import tools
import plot_confusion_matrix_Copy1 as plot_confusion_matrix
from sklearn.metrics import confusion_matrix

training_data, training_label, validation_data, validation_label, validation_cate_label = tools.get_data()

ks = 5
num_layer = 10
bs = 30
lr = 0.001

Using TensorFlow backend.


Train Data: (10151, 9000, 1)
Train Label:  (10151, 4)
KFold(n_splits=5, random_state=None, shuffle=True)
trian: [    0     1     2 ... 10148 10149 10150] len 8120 test: [    5     7     9 ... 10133 10134 10143] len 2031
trian: [    0     2     5 ... 10147 10149 10150] len 8121 test: [    1     3     4 ... 10139 10145 10148] len 2030
trian: [    0     1     2 ... 10146 10147 10148] len 8121 test: [   12    13    20 ... 10140 10149 10150] len 2030
trian: [    1     3     4 ... 10148 10149 10150] len 8121 test: [    0     2    15 ... 10127 10137 10142] len 2030
trian: [    0     1     2 ... 10148 10149 10150] len 8121 test: [    6    11    33 ... 10144 10146 10147] len 2030


### lr = 0.001

In [2]:
def run(bs, lr, ks, num_layer):
    
    for index, (X_train, Y_train, X_val, Y_val, val_cat) in enumerate(zip(training_data,
                                                       training_label,
                                                       validation_data,
                                                       validation_label,
                                                       validation_cate_label)):
        
        model = tools.create_model(lr, bs, ks, num_layer)
        print("Fold "+str(index))
        early_stop = EarlyStopping(patience=20)
        history = model.fit(x = X_train, 
                            y = Y_train,
                            epochs=100,
                            validation_data=(X_val, Y_val),
                            callbacks=[early_stop],
                            batch_size=bs, 
                            verbose=1)
        evaluation = model.evaluate(x = X_val, y = Y_val)
        validation_prediction = model.predict_classes(X_val, batch_size=bs)
        score = f1_score(val_cat, validation_prediction, average=None)
        print(score)
        
#         test_prediction = model.predict_classes(X_val, batch_size=1)
#         cnf_matrix = confusion_matrix(val_cat, test_prediction)
#         plot_confusion_matrix.plot_confusion_matrix(cnf_matrix, classes=['AF','Noise','Normal','Other'], save_png=True)
        
    
run(bs, lr, ks, num_layer)

Fold 0
Train on 8120 samples, validate on 2031 samples
Epoch 1/100
8120/8120 [==============================] - 22s 3ms/step - loss: 1.0216 - acc: 0.5676 - val_loss: 0.9607 - val_acc: 0.5913
Epoch 2/100
8120/8120 [==============================] - 11s 1ms/step - loss: 0.8935 - acc: 0.6028 - val_loss: 0.8510 - val_acc: 0.6155
Epoch 3/100
8120/8120 [==============================] - 12s 1ms/step - loss: 0.7897 - acc: 0.6687 - val_loss: 0.7476 - val_acc: 0.6947
Epoch 4/100
8120/8120 [==============================] - 12s 1ms/step - loss: 0.7011 - acc: 0.7055 - val_loss: 0.6608 - val_acc: 0.7317
Epoch 5/100
8120/8120 [==============================] - 12s 1ms/step - loss: 0.6382 - acc: 0.7361 - val_loss: 0.6648 - val_acc: 0.7415
Epoch 6/100
8120/8120 [==============================] - 12s 1ms/step - loss: 0.6068 - acc: 0.7515 - val_loss: 0.5605 - val_acc: 0.7804
Epoch 7/100
8120/8120 [==============================] - 12s 1ms/step - loss: 0.5828 - acc: 0.7704 - val_loss: 0.6022 - val_acc: 

8121/8121 [==============================] - 11s 1ms/step - loss: 0.4132 - acc: 0.8524 - val_loss: 0.5052 - val_acc: 0.8251
Epoch 19/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.4058 - acc: 0.8543 - val_loss: 0.4656 - val_acc: 0.8438
Epoch 20/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3932 - acc: 0.8572 - val_loss: 0.5177 - val_acc: 0.8330
Epoch 21/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3809 - acc: 0.8673 - val_loss: 0.5014 - val_acc: 0.8271
Epoch 22/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3859 - acc: 0.8573 - val_loss: 0.4586 - val_acc: 0.8394
Epoch 23/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3708 - acc: 0.8660 - val_loss: 0.4318 - val_acc: 0.8507
Epoch 24/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3718 - acc: 0.8698 - val_loss: 0.6395 - val_acc: 0.7660
Epoch 25/100
8121/8121 [==============================

Epoch 34/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3228 - acc: 0.8835 - val_loss: 0.4600 - val_acc: 0.8448
Epoch 35/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3203 - acc: 0.8828 - val_loss: 0.5227 - val_acc: 0.8330
Epoch 36/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3209 - acc: 0.8854 - val_loss: 0.4547 - val_acc: 0.8419
Epoch 37/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3122 - acc: 0.8877 - val_loss: 0.4480 - val_acc: 0.8433
Epoch 38/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3155 - acc: 0.8900 - val_loss: 0.4367 - val_acc: 0.8438
Epoch 39/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3042 - acc: 0.8914 - val_loss: 0.5175 - val_acc: 0.8399
Epoch 40/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3027 - acc: 0.8929 - val_loss: 0.4588 - val_acc: 0.8463
Epoch 41/100
8121/8121 [=================

8121/8121 [==============================] - 12s 1ms/step - loss: 0.3532 - acc: 0.8745 - val_loss: 0.5005 - val_acc: 0.8340
Epoch 28/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.3429 - acc: 0.8819 - val_loss: 0.4472 - val_acc: 0.8399
Epoch 29/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.3342 - acc: 0.8803 - val_loss: 0.4307 - val_acc: 0.8438
Epoch 30/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.3348 - acc: 0.8831 - val_loss: 0.5690 - val_acc: 0.7956
Epoch 31/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.3479 - acc: 0.8803 - val_loss: 0.4204 - val_acc: 0.8512
Epoch 32/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.3304 - acc: 0.8804 - val_loss: 0.4305 - val_acc: 0.8557
Epoch 33/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.3203 - acc: 0.8867 - val_loss: 0.4439 - val_acc: 0.8414
Epoch 34/100
8121/8121 [==============================

Epoch 26/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3719 - acc: 0.8668 - val_loss: 0.4488 - val_acc: 0.8355
Epoch 27/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3555 - acc: 0.8776 - val_loss: 0.4595 - val_acc: 0.8310
Epoch 28/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3461 - acc: 0.8734 - val_loss: 0.4642 - val_acc: 0.8251
Epoch 29/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3486 - acc: 0.8769 - val_loss: 0.4556 - val_acc: 0.8458
Epoch 30/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3355 - acc: 0.8840 - val_loss: 0.4642 - val_acc: 0.8483
Epoch 31/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3242 - acc: 0.8843 - val_loss: 0.4777 - val_acc: 0.8281
Epoch 32/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3253 - acc: 0.8870 - val_loss: 0.4697 - val_acc: 0.8360
Epoch 33/100
8121/8121 [=================

### lr = 0.0001

In [2]:
def run(bs, lr, ks, num_layer):
    
    for index, (X_train, Y_train, X_val, Y_val, val_cat) in enumerate(zip(training_data,
                                                       training_label,
                                                       validation_data,
                                                       validation_label,
                                                       validation_cate_label)):
        
        model = tools.create_model(0.0001, bs, ks, num_layer)
        print("Fold "+str(index))
        early_stop = EarlyStopping(patience=20)
        history = model.fit(x = X_train, 
                            y = Y_train,
                            epochs=100,
                            validation_data=(X_val, Y_val),
                            callbacks=[early_stop],
                            batch_size=bs, 
                            verbose=1)
        evaluation = model.evaluate(x = X_val, y = Y_val)
        validation_prediction = model.predict_classes(X_val, batch_size=bs)
        score = f1_score(val_cat, validation_prediction, average=None)
        print(score)
        
#         test_prediction = model.predict_classes(X_val, batch_size=1)
#         cnf_matrix = confusion_matrix(val_cat, test_prediction)
#         plot_confusion_matrix.plot_confusion_matrix(cnf_matrix, classes=['AF','Noise','Normal','Other'], save_png=True)
        
    
run(bs, lr, ks, num_layer)

Fold 0
Train on 8120 samples, validate on 2031 samples
Epoch 1/100
8120/8120 [==============================] - 19s 2ms/step - loss: 1.1812 - acc: 0.5133 - val_loss: 1.0720 - val_acc: 0.5731
Epoch 2/100
8120/8120 [==============================] - 11s 1ms/step - loss: 1.0196 - acc: 0.5805 - val_loss: 1.0524 - val_acc: 0.5731
Epoch 3/100
8120/8120 [==============================] - 11s 1ms/step - loss: 0.9654 - acc: 0.5851 - val_loss: 1.0077 - val_acc: 0.5731
Epoch 4/100
8120/8120 [==============================] - 11s 1ms/step - loss: 0.9165 - acc: 0.5913 - val_loss: 0.9027 - val_acc: 0.6081
Epoch 5/100
8120/8120 [==============================] - 11s 1ms/step - loss: 0.8691 - acc: 0.6268 - val_loss: 0.8485 - val_acc: 0.6425
Epoch 6/100
8120/8120 [==============================] - 11s 1ms/step - loss: 0.8017 - acc: 0.6579 - val_loss: 0.7928 - val_acc: 0.6642
Epoch 7/100
8120/8120 [==============================] - 11s 1ms/step - loss: 0.7772 - acc: 0.6688 - val_loss: 0.7629 - val_acc: 

8121/8121 [==============================] - 11s 1ms/step - loss: 0.7873 - acc: 0.6573 - val_loss: 0.7800 - val_acc: 0.6798
Epoch 8/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7498 - acc: 0.6757 - val_loss: 0.7105 - val_acc: 0.6842
Epoch 9/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7412 - acc: 0.6812 - val_loss: 0.6913 - val_acc: 0.6916
Epoch 10/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7216 - acc: 0.6878 - val_loss: 0.7597 - val_acc: 0.6842
Epoch 11/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.6890 - acc: 0.7055 - val_loss: 0.7092 - val_acc: 0.7015
Epoch 12/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.6522 - acc: 0.7217 - val_loss: 0.6615 - val_acc: 0.7108
Epoch 13/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.6242 - acc: 0.7345 - val_loss: 0.6386 - val_acc: 0.7355
Epoch 14/100
8121/8121 [==============================] 

Epoch 18/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.4981 - acc: 0.8123 - val_loss: 0.5597 - val_acc: 0.7906
Epoch 19/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.4815 - acc: 0.8187 - val_loss: 0.5220 - val_acc: 0.8025
Epoch 20/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.4639 - acc: 0.8228 - val_loss: 0.5496 - val_acc: 0.7980
Epoch 21/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.4526 - acc: 0.8340 - val_loss: 0.5115 - val_acc: 0.8187
Epoch 22/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.4427 - acc: 0.8354 - val_loss: 0.5367 - val_acc: 0.8133
Epoch 23/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.4276 - acc: 0.8430 - val_loss: 0.5233 - val_acc: 0.7956
Epoch 24/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.4291 - acc: 0.8419 - val_loss: 0.5116 - val_acc: 0.8187
Epoch 25/100
8121/8121 [=================

Epoch 30/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3964 - acc: 0.8570 - val_loss: 0.4616 - val_acc: 0.8291
Epoch 31/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3941 - acc: 0.8569 - val_loss: 0.4499 - val_acc: 0.8286
Epoch 32/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.3864 - acc: 0.8596 - val_loss: 0.4572 - val_acc: 0.8271
Epoch 33/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.3827 - acc: 0.8626 - val_loss: 0.4556 - val_acc: 0.8286
Epoch 34/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.3816 - acc: 0.8633 - val_loss: 0.4469 - val_acc: 0.8345
Epoch 35/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.3655 - acc: 0.8670 - val_loss: 0.4625 - val_acc: 0.8286
Epoch 36/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.3558 - acc: 0.8701 - val_loss: 0.4821 - val_acc: 0.8320
Epoch 37/100
8121/8121 [=================

Epoch 35/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.3673 - acc: 0.8666 - val_loss: 0.4471 - val_acc: 0.8379
Epoch 36/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.3670 - acc: 0.8713 - val_loss: 0.4388 - val_acc: 0.8350
Epoch 37/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.3682 - acc: 0.8680 - val_loss: 0.4547 - val_acc: 0.8419
Epoch 38/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.3558 - acc: 0.8733 - val_loss: 0.4071 - val_acc: 0.8527
Epoch 39/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3464 - acc: 0.8797 - val_loss: 0.4127 - val_acc: 0.8507
Epoch 40/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.3355 - acc: 0.8796 - val_loss: 0.4200 - val_acc: 0.8399
Epoch 41/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.3318 - acc: 0.8809 - val_loss: 0.4349 - val_acc: 0.8443
Epoch 42/100
8121/8121 [=================

confusion matrix(hidden)

In [2]:
import numpy as np
import plot_confusion_matrix_Copy1 as plot_confusion_matrix
cnf_matrix = np.array([[142, 3, 3, 21],[8, 1127, 15, 65], [6, 7, 35, 5], [27, 119, 14, 434]])
plot_confusion_matrix.plot_confusion_matrix(cnf_matrix, classes=['AF','Normal','Noisy','Other'], save_png=True)